# Menggunakan Pembelajaran Mesin Otomatis

Ada banyak jenis algoritme pembelajaran mesin yang dapat Anda gunakan untuk melatih model, dan terkadang tidak mudah untuk menentukan algoritme yang paling efektif untuk persyaratan data dan prediksi Anda. Selain itu, Anda dapat secara signifikan memengaruhi performa prediktif model dengan melakukan pra-pemrosesan data pelatihan, menggunakan teknik seperti normalisasi, imputasi fitur yang hilang, dan lainnya. Dalam upaya Anda untuk menemukan model *terbaik* untuk kebutuhan Anda, Anda mungkin perlu mencoba banyak kombinasi algoritme dan transformasi prapemrosesan; yang memakan banyak waktu dan sumber daya komputasi.

Azure Machine Learning memungkinkan Anda untuk mengotomatiskan perbandingan model yang dilatih menggunakan algoritme dan opsi prapemrosesan yang berbeda. Anda dapat menggunakan antarmuka visual di [Azure Machine Learning studio](https://ml/azure.com) atau SDK untuk memanfaatkan kemampuan ini. SDK memberi Anda kontrol yang lebih besar atas setelan untuk eksperimen pembelajaran mesin otomatis, tetapi antarmuka visual lebih mudah digunakan.

## Sebelum memulai

Selain versi terbaru paket **azureml-sdk** dan **azureml-widgets**, Anda memerlukan paket **azureml-train-automl** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa itu diinstal.

In [ ]:
pip show azureml-train-automl

## Menghubungkan ke ruang kerja Anda

Dengan menginstal paket SDK yang diperlukan, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Mempersiapkan data

Anda tidak perlu membuat skrip pelatihan untuk pembelajaran mesin otomatis, tetapi Anda perlu memuat data pelatihan. Dalam hal ini, Anda akan menggunakan set data yang berisi detail pasien diabetes, lalu membaginya menjadi dua set data: satu untuk pelatihan, dan satu lagi untuk validasi model.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# Split the dataset into training and validation subsets
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

## Menyiapkan komputasi

Salah satu manfaat komputasi awan adalah skalanya sesuai permintaan, memungkinkan Anda menyediakan sumber daya komputasi yang cukup untuk memproses beberapa percobaan pembelajaran mesin otomatis yang dijalankan oleh beberapa anak secara paralel.

Gunakan kode berikut untuk menentukan kluster komputasi Azure Machine Learning (akan dibuat jika belum ada)..

> **Penting**: Ubah *your-compute-cluster* menjadi nama kluster komputasi Anda dalam kode di bawah ini sebelum menjalankannya! Nama kluster harus berupa nama unik global dengan panjang antara 2 hingga 16 karakter. Karakter yang valid adalah huruf, angka, dan - karakter.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **Catatan**: Instans dan kluster komputasi didasarkan pada gambar mesin virtual Azure standar. Untuk latihan ini, gambar *Standard_DS11_v2* direkomendasikan untuk mencapai keseimbangan optimal antara biaya dan performa. Jika langganan Anda memiliki kuota yang tidak menyertakan gambar ini, pilih gambar alternatif; tetapi perlu diingat bahwa gambar yang lebih besar mungkin memerlukan biaya lebih tinggi dan gambar yang lebih kecil mungkin tidak cukup untuk menyelesaikan tugas. Atau, minta administrator Azure Anda untuk memperbesar kuota Anda.

## Mengonfigurasi pembelajaran mesin otomatis

Sekarang Anda siap mengonfigurasi eksperimen pembelajaran mesin otomatis.

Salah satu pengaturan konfigurasi yang paling penting adalah metrik yang performa modelnya harus dievaluasi. Anda dapat mengambil daftar metrik yang dihitung dengan pembelajaran mesin otomatis untuk jenis tugas model tertentu (klasifikasi atau regresi) seperti ini:

In [ ]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

Setelah memutuskan metrik yang ingin Anda optimalkan (dalam contoh ini, *AUC_weighted*), Anda dapat mengonfigurasi proses pembelajaran mesin otomatis. Untuk melakukannya, Anda memerlukan konfigurasi AutoML yang menentukan metrik target serta opsi seperti data yang akan digunakan, berapa banyak kombinasi yang harus dicoba, dan sebagainya.

> **Catatan**: Dalam contoh ini, Anda akan membatasi eksperimen menjadi 4 perulangan untuk mengurangi jumlah waktu yang dibutuhkan. Pada kenyataannya, Anda mungkin akan mencoba lebih banyak perulangan.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## Menjalankan eksperimen pembelajaran mesin otomatis

Oke, Anda siap melanjutkan. Mari kita jalankan eksperimen pembelajaran mesin otomatis.

> **Catatan**: Ini mungkin memakan waktu lama!

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-diabetes-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Melihat detail eksekusi turunan

Saat eksperimen selesai, lihat output di widget, dan klik proses yang menghasilkan hasil terbaik untuk melihat detailnya.

Kemudian klik tautan untuk melihat detail eksperimen di studio Azure Machine Learning dan lihat detail eksperimen keseluruhan sebelum melihat detail untuk masing-masing proses yang menghasilkan hasil terbaik. Ada banyak informasi di sini tentang performa model yang dihasilkan.

Anda juga dapat mengambil semua turunan yang berjalan dan melihat metriknya menggunakan SDK:

In [ ]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

## Mendapatkan eksekusi terbaik

Anda dapat mengambil eksekusi dengan performa terbaik, dan melihat detailnya seperti ini:

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Akhirnya, setelah menemukan model berperforma terbaik, Anda dapat mendaftarkannya.

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **Informasi Lebih Lanjut**: Untuk informasi selengkapnya Pembelajaran Mesin Otomatis, lihat [dokumentasi Azure ML](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train).